# Lecture 9:  A simple machine learning algorithm

## K nearest neighbors (KNN) algorithm fundamentals, implementation and evaluation

### Agenda

1. Main idea
2. Implementation
3. Parameter(K) selection and the query of nearest neighbors
4. Disadvantages
5. Result evaluation: confusion matrix
6. Use KNN in sklearn

### 1. KNN main idea
1. A type of instance-based learning: store instances of training data rather than learn a internal model
2. Predict the label/attribution of new data by using the **K nearest data points** in the training set
2. **K** can be a user-defined constant (k-nearest neighbor learning), or vary based on the local density of points
3. Standard Euclidean distance is the most common distance metric used in KNN

![KNN](KNN.png)   

## 2. Implementation
    
1. Data preparation
    1. Load data: `ds.load_iris()`
    2. Prepare the training set and test set: `train_test_split(X, y, test_size = 0.3)`
    
2. Model training `fit()`
    1. No explicit training step in KNN
    2. The training phase of KNN is to load the feature vectors and class labels of the training samples
    
3. Prediction and evaluation `predict()`
    1. Find k neighbors for a data sample
    2. vote to determine the class label
    
4. Model persistance `pickle.dump()` and `pickle.load()`
    1. No explicit model trained
    2. The persistance step is to store the training samples

In [7]:
import numpy as np

# two feature vectors
x = np.array([1, 0, 0])
y = np.array([0, 1, 1])
# calculate the euclidean distance
# sqrt((x1 - y1)^2) + ... + (xn-yn)^2)
d = np.sqrt(np.sum((x - y) ** 2))
print(d)

1.7320508075688772


In [26]:
import numpy as np
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split

class MyKNN:
    '''My implementation of the KNN algorithm based on euclidean distance.
    '''
    
    def __init__(self, k = 5):
        ''' only need to initialize KNN
        
            input:
                k: the number of nearest neighbors to search
        '''
        self.K = k
        
    def fit(self, X_train, y_train):
        '''The training phase of KNN is to load the feature vectors and class labels of the training samples
        '''
        self.X_train = X_train
        self.y_train = y_train
   
    def predict(self, X):
        ''' Find k neighbors for a data sample, and vote to determine the class label
            
            input:
                X: input feature vectors 
                
            return:
                pred: predictions for X
        '''
        
        n = X.shape[0]
        pred = np.zeros(n, dtype = int)
        
        for i in range(n): # 0, ..., n-1
            nns = self.findKNgbs(X[i, :]) # nns, indices of K nearest neighbors
            labels = self.y_train[nns] # class labels
            
            # vote to choose the label with more counts (ex: k=5 choose the label with 3+)
            cnts = np.bincount(labels) # voting by counting labels in each category
            pred[i] = np.argmax(cnts) # use the label with the max counts as the predicted label
        
        return pred 

    def findKNgbs(self, x):
        '''find neighbors using euclidean distance. Euclidean distance between x1 and x2 is squrt((x1-x2)^2)
            
            input:
                x: a single data sample(1*4)
                
            return:
                the k indices for the k nearest neighbors of x
        '''
        distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1)) # dimensionality is n x 1
        indices = np.argsort(distances) # indices of sorted array
        return indices[:self.K]

    def errRate(self, real, pred, precison = 2):
        err = 0
        n = real.shape[0]
        err = np.sum(real != pred)
        return round(err/n, precison)


#1. data preparation
iris = ds.load_iris()
#dir(iris)
X = iris['data']
y = iris['target']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(150, 4) (150,)
(120, 4) (120,) (30, 4) (30,)


In [28]:
# 2. model training
knn = MyKNN(k=5)
knn.fit(X_train, y_train)

# 3. prediction and evaluation
# training error
y_train_pred = knn.predict(X_train)
train_err = knn.errRate(y_train, y_train_pred)
print('training error rate:', train_err)
# test error
y_test_pred = knn.predict(X_test)
test_err = knn.errRate(y_test, y_test_pred)
print('test error rate:', test_err)

training error rate: 0.05
test error rate: 0.03


In [22]:
import pickle
# using pickle.dump() to save model
with open('knn.pickle', 'wb') as file:
    pickle.dump(knn, file)

In [7]:
# using pickle.load() to save model
with open('knn.pickle', 'rb') as file:
    knn1 = pickle.load(file)

## 3. Parameter(K) selection and the choice of nearest neighbors
1. The best choice of K depends upon the data
    1. small k make the algorithm sensitive to noise
    2. large k reduces the effect of noise, but make boundaries between classes blurr
    3. Search k: k = 1, 3, 5, 7, 9


In [30]:
import time

# 1. parameter selection
knn2 = MyKNN(2)
knn2.fit(X_train, y_train)
train_pred = knn2.predict(X_train)
print('Error rate on training set: {}%'.format(100*knn2.errRate(y_train, train_pred)))

test_pred = knn2.predict(X_test)
print('Error rate on test set: {}%'.format(100*knn2.errRate(y_test, test_pred)))


# time
start = time.time()
train_pred = knn2.predict(X_train)
end = time.time()
print(end - start, 'seconds')


Error rate on training set: 2.0%
Error rate on test set: 3.0%
0.004998683929443359 seconds


## 4. Pros and Cons

###  Pros
1. It can be applied to both regression and classification problems
2. Good for problems with very irregular dicision bnoundary

### Cons    
1. Compute the distance and sort all the training data can be very slow if there are a large number of training examples
2. Every sample contributes equally --> citation-KNN (https://papers.nips.cc/paper/1346-a-framework-for-multiple-instance-learning.pdf)
3. Sensitive to K
4. Not robust to noise -->  implement learning based on the number of neighbors within a fixed radius r of each training poin

## 5. Result evaluation: confusion matrix

![confusion matrix](cfm.png)  

```python
from sklearn.metrics import confusion_matrix as cfm

m = cfm(y_train, train_pred)
print(m)

```

In [12]:
from sklearn.metrics import confusion_matrix as cfm


## 6. Use KNN in sklearn

In [15]:
from sklearn.neighbors import KNeighborsClassifier

ngh = KNeighborsClassifier(n_neighbors=3)
ngh.fit(X_train, y_train) 


KNeighborsClassifier(n_neighbors=3)